In [57]:
import pandas as pd
import json
import plotly.express as px

df_position = pd.read_json("data/raw/ais_position.json")
df_static = pd.read_json("data/raw/ais_static.json")
df_types = pd.read_csv("data/raw/ais_ShipTypes.csv", sep=";")
df_status = pd.read_csv("data/raw/ais_NavigationStatus.csv", sep=";")

df_static['timestamp'] = pd.to_datetime(df_static['timestamp'].str.replace('UTC', ''), format='ISO8601')
df_static.sort_values('MMSI')

,timestamp,MMSI,Destination,Dimension,Eta,Type
289,2025-06-19 07:34:44.967628467+00:00,2249955,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 0, 'Minute': 0, 'Month': 0}",99
160,2025-06-19 07:34:37.533935342+00:00,111224501,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",51
383,2025-06-19 07:34:48.772067177+00:00,111232537,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",51
335,2025-06-19 07:34:47.020353291+00:00,111244515,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",0
1135,2025-06-19 07:35:24.659942052+00:00,203999303,,"{'A': 12, 'B': 8, 'C': 5, 'D': 1}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",55
...,...,...,...,...,...,...
1199,2025-06-19 07:35:28.362551229+00:00,710999915,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 1, 'Hour': 0, 'Minute': 0, 'Month': 1}",3
880,2025-06-19 07:35:12.477566121+00:00,730151081,,"{'A': 12, 'B': 18, 'C': 5, 'D': 5}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",52
60,2025-06-19 07:34:32.030481735+00:00,730151802,LA GLORIA,"{'A': 15, 'B': 15, 'C': 6, 'D': 6}","{'Day': 2, 'Hour': 0, 'Minute': 0, 'Month': 9}",31
1811,2025-06-19 07:35:57.266158145+00:00,730153873,BARRANQUILLA,"{'A': 7, 'B': 33, 'C': 5, 'D': 7}","{'Day': 10, 'Hour': 18, 'Minute': 0, 'Month': 9}",90


In [58]:
# Keep LATEST MMSI static data
df_static_latest = df_static.sort_values("timestamp").drop_duplicates("MMSI", keep="last")
# Add the Ship Type string
df_static_latest = pd.merge(df_static_latest, df_types, on='Type', how='left')
# Calculate Lenght and Width
df_static_latest["Length"] = df_static_latest["Dimension"].apply(lambda x: x.get("A", 0) + x.get("B", 0))
df_static_latest["Width"] = df_static_latest["Dimension"].apply(lambda x: x.get("C", 0) + x.get("D", 0))

# Join Navigation Status to position
df_position_final = pd.merge(df_position, df_status, on='NavigationalStatus', how='left')

# final_df
final_df = pd.merge(df_position_final, df_static_latest, on='MMSI', how='left', suffixes = ('_position', '_static'))
final_df

# QUESTION --> EST CE QUE JE GARDE TOUTES LES POISTIONS ? SI OUI AJOUTER UN POSITIONSTATUS : 1 = Latest, 0 = Old

,timestamp_position,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,...,timestamp_static,Destination,Dimension,Eta,Type,Type Description,Category,SubCategory,Length,Width
0,2025-06-19 07:34:29.35955696 +0000 UTC,247386800,ML FREYJA,48.361797,-6.203332,216.4,0,-8,13.3,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-19 07:34:29.36564606 +0000 UTC,219499000,TORM EVELYN,49.948423,-4.134202,283.1,0,0,7.9,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-06-19 07:34:29.375062186 +0000 UTC,232042335,ERRINGTON,51.493135,-0.122355,311.9,15,14,0.0,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-06-19 07:34:29.377760254 +0000 UTC,368052910,LIL EMI,29.468355,-94.972238,95.5,15,-128,0.0,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-06-19 07:34:29.385482459 +0000 UTC,244620704,LUTIN,51.601112,3.971102,59.5,0,127,7.8,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,2025-06-19 07:34:49.140866035 +0000 UTC,246400000,ANCHORAGE,51.226658,2.933237,237.5,0,0,0.0,0,...,2025-06-19 07:34:29.356405375+00:00,BE OSTEND,"{'A': 13, 'B': 93, 'C': 12, 'D': 4}","{'Day': 19, 'Hour': 6, 'Minute': 30, 'Month': 6}",33.0,Dredging or underwater ops,Dredging,NaN,106.0,16.0
2506,2025-06-19 07:34:49.146606673 +0000 UTC,367612050,BRANDY NICOLE,29.946733,-93.833025,360.0,0,-128,0.1,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2507,2025-06-19 07:34:49.149349062 +0000 UTC,368217380,MFS SHIFTER,30.080498,-94.085287,75.5,15,-128,0.0,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2508,2025-06-19 07:34:49.150656735 +0000 UTC,338768000,SWEENEY,29.993067,-93.909562,299.5,12,0,0.0,0,...,2025-06-19 07:35:39.589241884+00:00,TEXAS CITY,"{'A': 104, 'B': 12, 'C': 9, 'D': 9}","{'Day': 18, 'Hour': 4, 'Minute': 0, 'Month': 1}",57.0,Spare - Local Vessel,Spare,NaN,116.0,18.0


In [59]:
fig = px.scatter_map(final_df, lat="lat", lon="lon", color="NavigationalStatus",
                  color_continuous_scale=px.colors.cyclical.IceFire)

fig.show()